In [132]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import face_recognition_models
import dlib
import os
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates
from time import time
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
import face_recognition
import shutil

In [133]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

In [134]:
landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]


In [135]:
mp_drawing = mp.solutions.drawing_utils

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,min_detection_confidence=0.5)

### MP + Dlib utils


In [136]:
def landmark_pt_list(mesh_results,width,height):
    landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
    landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
    landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]

    if mesh_results.multi_face_landmarks:
        for i,face_landmarks in enumerate(mesh_results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5_1:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    landmark_point=dlib.point([x,y])
                    raw_landmark_set.append(landmark_point)
                    #display(landmark_point)
                all_points=dlib.points(raw_landmark_set)
#         return dlib.points([{
#             "nose_tip": [raw_landmark_set[0]],
#             "left_eye": raw_landmark_set[1:3],
#             "right_eye": raw_landmark_set[3:],
#         }])
        return all_points

def bounding_rect(detection_results,width,height):
    if detection_results.detections:
        for i,detection in enumerate(detection_results.detections):
            if i==0:
                # bbox data
                bbox = detection.location_data.relative_bounding_box
                xy_min = _normalized_to_pixel_coordinates(bbox.xmin, bbox.ymin, height,width)
                xy_max = _normalized_to_pixel_coordinates(bbox.xmin + bbox.width, bbox.ymin + bbox.height,height,width)
                if xy_min is None or xy_max is None:
                    #print("face out of frame")
                    return
                else:
                    xmin,ymin =xy_min
                    xmax,ymax = xy_max
                    #bbox_points = { "xmin" : xmin,"ymin" : ymin,"xmax" : xmax,"ymax" : ymax}
                    rectangle= dlib.rectangle(left=xmin, top=ymax, right=xmax, bottom=ymin)
                    return rectangle

def ret_encoding(filepath,num_jitters=1):
    #image_input = cv2.imread(filepath)
    #image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
    image_input =face_recognition.load_image_file(filepath)
    width,height=image_input.shape[:-1]
    detection_results = face_detection.process(image_input)
    mesh_results = face_mesh.process(image_input)
    
    all_points=  landmark_pt_list(mesh_results,width,height)  
    b_box=bounding_rect(detection_results,width,height)
    if (all_points is None) or (b_box is None):
        return 
    raw_landmark_set=dlib.full_object_detection(b_box,all_points)
    #display(all_points)
    #display(b_box)
    encodings=face_encoder.compute_face_descriptor(image_input, raw_landmark_set, num_jitters)

    return encodings

### Linear sorting

In [137]:
def get_img_list(folder):
    img_list=[]
    for file in os.listdir(folder):
        if not file.startswith('.') and os.path.isfile(os.path.join(folder, file)):
            filepath = os.path.join(folder, file)
            filepath=filepath.replace('\\' , '/')
            img_list.append(file)
    return img_list        
    print("got image list")

In [138]:
def get_d(enc1, enc2):
    enc1=np.array(enc1)
    enc2=np.array(enc2)
    d=np.linalg.norm(enc1 - enc2, axis=0)
    return d

In [139]:
# not df
def encode_list(folder, img_list):
    enc_dict={}
    for img in img_list:
        img_path = os.path.join(folder,img)
        encoding=(ret_encoding(img_path))
        enc_dict[img]=encoding 
    return enc_dict

In [140]:
# not df
def get_closest(folder, start_img, img_list, enc_dict):
    img_list.remove(start_img)
#     enc1=(ret_encoding(os.path.join(folder,start_img)))
    enc1=enc_dict[start_img]
    dist=[]
    dist_dict={}
    for img in img_list:
#         file2=filebase+str(i)+".jpg".replace('\\' , '/')
#         test_img = os.path.join(folder,img)
#         enc2=(ret_encoding(test_img))
        enc2 = enc_dict[img]
        if (enc1 is not None) and (enc2 is not None):
#             print(file2)
            d = get_d(enc1, enc2)
#             print(d)
            dist.append(d)
            dist_dict[d]=img
    dist.sort()
    print(len(dist))
    return dist[0], dist_dict[dist[0]]

In [179]:
def save_sorted(folder, image, counter, dist):
    sorted_name = "linear_sort_"+str(counter)+"_"+str(round(dist, 2))+".jpg"
    sortfolder="sorted2"
    newfolder = os.path.join(folder,sortfolder)
    old_name=os.path.join(folder,image)
    new_name=os.path.join(newfolder,sorted_name)
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)
    shutil.copy(old_name, new_name)
    print('saved, ',sorted_name)


### dataframe creation and sorting

In [180]:
def encode_list_df(folder, img_list):
#     enc_dict={}
    csv_name="face_encodings.csv"
    col1="file_name"
    col2="encoding"
    curr=0
    total = len(img_list)

    # encodings column list for splitting
    col_list=[]
    for i in range(128):
        col_list.append(col2+str(i))

    #initializing the dataframe
    image_data=pd.DataFrame(columns=[col1, col2])

    
    for img in img_list:
        if curr%10==0:print(curr,"/",total)
        curr+=1
        filepath = os.path.join(folder,img)        
        filepath=filepath.replace('\\' , '/')  ## cv2 accepts files with "/" instead of "\"
        encodings=ret_encoding(filepath)
        if encodings is not None:              ## checking if a face is found
            data=pd.DataFrame({col1:img,col2:[np.array(encodings)]})
            image_data = pd.concat([image_data,data],ignore_index=True)  

    #splitting the encodings column
    output_data = pd.DataFrame(image_data[col2].to_list(), columns=col_list)
    #adding the filename column and then puting it first
    output_data[[col1]]=pd.DataFrame(image_data[col1].tolist(),index=image_data.index)
    clms = output_data.columns.tolist()
    clms = clms[-1:] + clms[:-1]
    output_data=output_data[clms]
    # saving without index
    output_data.to_csv(csv_name, index=False)
    df = pd.read_csv(csv_name)
    
    return output_data

In [181]:
def get_closest_df(folder, start_img, df_enc):
    if start_img == "median":
        enc1 = df_enc.median().to_list()
#         print("in median")
    else:
#         enc1 = get 2-129 from df via stimg key
        enc1 = df_enc.loc[start_img].to_list()
        df_enc=df_enc.drop(start_img)
#         print("in new img",len(df_enc.index))
    
#     img_list.remove(start_img)
#     enc1=enc_dict[start_img]
    
    dist=[]
    dist_dict={}
    for index, row in df_enc.iterrows():
#         print(row['c1'], row['c2'])
#     for img in img_list:
        enc2 = row
        if (enc1 is not None) and (enc2 is not None):
            d = get_d(enc1, enc2)
            dist.append(d)
            dist_dict[d]=index
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]], df_enc

In [185]:
# test if new and old make a face
def is_face(image):
    # For static images:
    # I think this list is not used
    IMAGE_FILES = []
    with mp_face_detection.FaceDetection(model_selection=1, 
                                        min_detection_confidence=0.6
                                        ) as face_detection:
        # image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
#         detection_results = face_detection.process(image)

        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            is_face = False
        else:
            is_face = True
        # annotated_image = image.copy()
        # for detection in results.detections:
        #     is_face = True
        #     print('Nose tip:')
        #     print(mp_face_detection.get_key_point(
        #       detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
        #     mp_drawing.draw_detection(annotated_image, detection)
        # cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

        return is_face


In [186]:
# test if new and old make a face
def test_pair(last_file, new_file):
    try:
        img = cv2.imread(new_file)
        height, width, layers = img.shape
        size = (width, height)
        print('loaded img 1')
        
        last_img = cv2.imread(new_file)
        last_height, last_width, last_layers = last_img.shape
        last_size = (last_width, last_height)
        print('loaded img 2')
        
        # test to see if this is actually an face, to get rid of blank ones/bad ones
        if is_face(img):
            print('new file is face')
            # if not the first image
#             if i>0:
            # blend this image with the last image
            blend = cv2.addWeighted(img, 0.5, last_img, 0.5, 0.0)
            print('blended faces')
            blended_face = is_face(blend)
            print('is_face ',blended_face)
            # if blended image has a detectable face, append the img
            if blended_face:
#                     img_array.append(img)
                print('is a face! adding it')
                return True
            else:
                print('skipping this one')
                return False
            # for the first one, just add the image
            # this may need to be refactored in case the first one is bad?
#             else:
#                 print('this is maybe the first round?')
#                 img_array.append(img)
        else:
            print('new_file is not face: ',new_file)
            return False

#         i+=1

    except:
        print('failed:',new_file)
        return False


In [188]:
folder="/Users/michaelmandiberg/Documents/projects-active/facemap_production/images1674272442.9258912/"
img_list = get_img_list(folder)
# start_img = img_list[1]
start_img = "median"

# enc_dict = encode_list(folder, img_list)
df_enc = encode_list_df(folder, img_list)

df_enc.set_index('file_name', inplace=True)

# print(df_enc.median().to_list())

#with lists/dicts
# dist=0
# for i in range(len(img_list)-1):
#     save_sorted(folder, start_img, i, dist)
#     dist, start_img = get_closest(folder, start_img,img_list, enc_dict)
    
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         continue

#with df
dist=0
for i in range(len(df_enc.index)-2):
    dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
    if i>0:
        #test blend
#         last_file = os.path.join(folder,)
        blend_is_face = (test_pair(os.path.join(folder,last_img), os.path.join(folder,start_img)))
        print('blend_is_face ',blend_is_face)
        if blend_is_face:
#         print(test_pair(last_img,start_img))
            save_sorted(folder, start_img, i, dist)
            last_img = start_img
#         else:
#             start_img = last_img
            
    print(i)
    print(len(df_enc.index))
    print(dist)
    print (start_img)
    
    if dist > .37: 
        break
        
       

    

0 / 610
10 / 610
20 / 610
30 / 610
40 / 610
50 / 610
60 / 610
70 / 610
80 / 610
90 / 610
100 / 610
110 / 610
120 / 610
130 / 610
140 / 610
150 / 610
160 / 610
170 / 610
180 / 610
190 / 610
200 / 610
210 / 610
220 / 610
230 / 610
240 / 610
250 / 610
260 / 610
270 / 610
280 / 610
290 / 610
300 / 610
310 / 610
320 / 610
330 / 610
340 / 610
350 / 610
360 / 610
370 / 610
380 / 610
390 / 610
400 / 610
410 / 610
420 / 610
430 / 610
440 / 610
450 / 610
460 / 610
470 / 610
480 / 610
490 / 610
500 / 610
510 / 610
520 / 610
530 / 610
540 / 610
550 / 610
560 / 610
570 / 610
580 / 610
590 / 610
600 / 610
0
609
0.19910015689042002
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_594.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_1_0.13.jpg
1
608
0.12630849282404943
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_496.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is 

is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_33_0.25.jpg
33
576
0.2541672409903279
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_276.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_34_0.28.jpg
34
575
0.2757100566447882
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_127.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_35_0.24.jpg
35
574
0.24227519418067672
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_231.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_36_0.22.jpg
36
573
0.2182595010554797
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_381.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True


new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_68_0.25.jpg
68
541
0.2479959541109685
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_602.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_69_0.26.jpg
69
540
0.26122376406024395
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_417.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_70_0.14.jpg
70
539
0.14002688349948564
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_390.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_71_0.25.jpg
71
538
0.2545141389814399
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_436.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face

new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_103_0.25.jpg
103
506
0.2493172658848864
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_143.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_104_0.26.jpg
104
505
0.25954181155425854
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_535.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_105_0.25.jpg
105
504
0.2508793555162941
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_403.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_106_0.26.jpg
106
503
0.2568746618577233
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_264.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is

new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_139_0.15.jpg
139
470
0.15318995161997287
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_336.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_140_0.2.jpg
140
469
0.20189307792991032
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_180.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_141_0.2.jpg
141
468
0.20016867982325573
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_209.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_142_0.25.jpg
142
467
0.2510401355640922
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_449.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is

...

is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_244_0.27.jpg
244
365
0.27365554629453626
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_401.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_245_0.34.jpg
245
364
0.3350805052052949
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_65.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_246_0.31.jpg
246
363
0.30562597676252373
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_317.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_247_0.33.jpg
247
362
0.33381759721410614
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_248.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_fa

new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_279_0.29.jpg
279
330
0.2936250978812008
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_465.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_280_0.31.jpg
280
329
0.30864007968087465
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_453.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_281_0.28.jpg
281
328
0.28199642299590993
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_162.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_282_0.27.jpg
282
327
0.26991324810134687
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_567.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True


loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_314_0.28.jpg
314
295
0.28440174694329784
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_228.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_315_0.29.jpg
315
294
0.2942577257892495
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_492.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_316_0.27.jpg
316
293
0.268276104192155
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_458.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_317_0.29.jpg
317
292
0.2856247032465554
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_486.jpg
loaded img 1
loaded img 2
new file is face
blend

new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_351_0.34.jpg
351
258
0.34196282478272827
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_388.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_352_0.29.jpg
352
257
0.2921097582722468
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_252.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_353_0.35.jpg
353
256
0.34883492110508196
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_2.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_354_0.3.jpg
354
255
0.2954346732019623
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_282.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a

new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_386_0.31.jpg
386
223
0.3138705684958282
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_498.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_387_0.31.jpg
387
222
0.3103124857232335
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_203.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_388_0.34.jpg
388
221
0.34335031599603827
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_273.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
is a face! adding it
blend_is_face  True
saved,  linear_sort_389_0.32.jpg
389
220
0.31691632961728605
faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct674_190.jpg
loaded img 1
loaded img 2
new file is face
blended faces
is_face  True
i